# Process Orders Data
  1. Ingest the data into the data lakehouse - bronze_orders
  2. Perform quality checks and transform the data as required - silver_orders_clean
  3. Explode the items array from the order object - silver_orders

![processing_orders_data.png](./processing_orders_data.png "processing_orders_data.png")

### 1. Ingest the data into the data lakehouse - bronze_orders

In [0]:
create or refresh streaming table bronze_orders
comment 'Raw orders data ingested from the source system'
tblproperties ("quality" = "bronze")
as
select *,
        _metadata.file_path as input_file_path,
        current_timestamp as ingestion_timestamp
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/orders/',
  'json',
  map('cloudFiles.inferColumnTypes', 'true')
);


### 2. Perform quality checks and transform the data as required - silver_orders_clean

In [0]:
create or refresh streaming table silver_orders_clean(
  constraint valid_customer_id expect (customer_id is not null) on violation fail update,
  constraint valid_order_id expect (order_id is not null) on violation fail update,
  constraint valid_order_status expect (order_status in ("Pending", "Shipped", "Canceled", "Completed")),
  constraint valid_payment_method expect (payment_method in ("Credit Card", "PayPal", "Bank Transfer"))
)
comment 'Cleaned Orders Data'
tblproperties ('quality' = 'silver')
as
select order_id,
       customer_id,
       cast(order_timestamp as timestamp) as order_timestamp,
       payment_method,
       items,
       order_status
from stream(LIVE.bronze_orders);

### 3. Explode the items array from the order object - silver_orders

In [0]:
create streaming table silver_orders
as
select order_id,
       customer_id,
       order_timestamp,
       payment_method,
       order_status,
       item.item_id,
       item.name as item_name,
       item.price as item_price,
       item.quantity as item_quantity,
       item.category as item_category
from (select order_id, 
            customer_id,
            order_timestamp,
            payment_method,
            order_status,
            explode(items) as item
from stream(live.silver_orders_clean)
);